<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=25% ALIGN="right">
</figure>

<p><div> 
<br><b>Probabilistic Design</b>
<br><b>CEGM2XXX</b>
<br> <i>This course is given at the faculty of Civil Engineering and Geosciences at the Technical University of Delft</i>
<br><b>Teaching team:</b>
<br><i>- Dr. Ir. R.C.Lanzafame </i>
<br><i>- Dr. P.M. Nasarre</i>
</div>
<br>
<br><b>Notebooks created by teaching assistant:</b>
<br><i>- Siemen Algra</i>
</div>

### Case study: Component Reliability Analysis, Determinsitic part
#### Floating and submerged tunnel, crossing the strait of Gibraltar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import fsolve # for plotting the LSF

import openturns as ot
import openturns.viewer as viewer
ot.Log.Show(ot.Log.NONE)

import numpy as np
import scipy.linalg as scp
import matplotlib.pylab as plt
import time

import sympy as sp
import scipy

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6




### 1. Load FEM model 
The material properties can be adjusted in the **FEM.py** file.

In [ ]:
# Import local classes
import sys
# sys.path.append(r'../../src/')

# Obtain function that calculates the stress
import FEM
# Function takes external forces as input and returns the stress

### 2. Define external forces depending on the random variables
Using the **Morison equation**

In [ ]:
def F_morison_swell_current(significant_wave_height_swell, U_current_veloctiy):
    # General properties
    rhoW = 1025                                                 # kg/m^3
    D = np.sqrt(38/np.pi)*2                                     # diameter of cylinder
    depth = 30                                                  # depth of water

    # Swell Wave properties
    Tp = 12
    Omega = 2*np.pi/Tp
    L = 9.81*Tp**2/(2*np.pi)
    k = 2*np.pi/L
    Amplitude = significant_wave_height_swell/2                 # wave amplitude

    
    # Water velocity
    cos_func = 1 #np.cos(omega*t-k*x)
    u_swell =  Amplitude*Omega*np.exp(-k*depth)*cos_func
    
    # Intertia force
    CM = 1+((9.2/2) / (20.8/2))                                 # added mass coefficient
    F_inertia_swell =  (np.pi*rhoW*CM*D**2)/4 * (u_swell)
    
    # Drag force
    CD = 0.13 # drag coefficient
    F_drag_swell = 0.5*rhoW*CD*D*(u_swell+U_current_veloctiy)**2
    
    F_combined_swell_current = np.sqrt(F_inertia_swell**2 + F_drag_swell**2)
    
    return F_combined_swell_current


In [ ]:
def F_morison_wind(significant_wave_height_windsea):
    # General properties
    rhoW = 1025                                                 # kg/m^3
    D = np.sqrt(38/np.pi)*2                                     # diameter of cylinder
    depth = 30                                                  # depth of water

    # Wind sea Wave properties
    Tp = 12
    Omega = 2*np.pi/Tp
    L = 9.81*Tp**2/(2*np.pi)
    k = 2*np.pi/L
    Amplitude = significant_wave_height_windsea/2                 # wave amplitude

    
    # Water velocity
    cos_func = 1 #np.cos(omega*t-k*x)
    u_wind =  Amplitude*Omega*np.exp(-k*depth)*cos_func
    
    # Intertia force
    CM = 1+((9.2/2) / (20.8/2))                                 # added mass coefficient
    F_inertia_wind =  (np.pi*rhoW*CM*D**2)/4 * (u_wind)
    
    # Drag force
    CD = 0.13 # drag coefficient
    F_drag_wind = 0.5*rhoW*CD*D*(u_wind)**2
    
    F_combined_wind = np.sqrt(F_inertia_wind**2 + F_drag_wind**2)
    
    return F_combined_wind

### 3. Define function to calculate axial stresses with global matrix

In [ ]:
def total_calculation_axial_stresses(significant_wave_height_swell, significant_wave_height_windsea, U_current_velocity):
    start_time = time.time()  # Start the timer
    
    # Calculate stress using FEM
    sigma_wind = FEM.calculate_stress_FEM(F_morison_wind(significant_wave_height_windsea))
    sigma_swell = FEM.calculate_stress_FEM(F_morison_swell_current(significant_wave_height_swell, U_current_velocity))
    
    # Total stress
    sigma = sigma_wind + sigma_swell 
    
    end_time = time.time()  # Stop the timer
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    print(f"Time to run FEM calculation: {elapsed_time:.2f} seconds")  # Print the elapsed time
    
    return sigma 

### 4. Run FEM model
Input forces and calculate axial stresses

In [ ]:
output_FEM = total_calculation_axial_stresses(significant_wave_height_swell=1,     # m
                                              significant_wave_height_windsea=1,   # m
                                              U_current_velocity=2                 # m/s
                                              )

In [ ]:
plt.plot(output_FEM);
plt.ylabel('Stress [MPa]')
plt.xlabel('Tunnel element [i]')
plt.title('Stress in tunnel elements');